## 1. 패키지 로드

In [13]:
import numpy as np
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
from tqdm.notebook import tqdm

## 2. 데이터 로드

In [2]:
actor_df = pd.read_excel('./text_data/Actor_image_info.xlsx')
actor_df.head(2)

,Name,age,Sex,photo,나무위키,배역내용,커뮤니티
0,감우성,1970,남,https://search.pstatic.net/common?type=b&size=...,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",['첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다. 남에게 피해 주지 않고 올곧...,NaN
1,강경준,1983,남,https://search.pstatic.net/common?type=b&size=...,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...","['흙수저? 아니, 몸짱 얼짱 마음까지 짱인 태권도장 사범. 얼굴만 봐도 신뢰감이 ...",NaN


## 3. 불용어 선정을 위한 데이터 전처리 및 분석

### 1) 데이터 전처리

In [6]:
# 내용이 아예 없는 배우 제거
actor_df = actor_df.dropna(subset=['나무위키', '배역내용', '커뮤니티'], how='all').reset_index(drop=True)
actor_df

,Name,age,Sex,photo,나무위키,배역내용,커뮤니티
0,감우성,1970,남,https://search.pstatic.net/common?type=b&size=...,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",['첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다. 남에게 피해 주지 않고 올곧...,NaN
1,강경준,1983,남,https://search.pstatic.net/common?type=b&size=...,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...","['흙수저? 아니, 몸짱 얼짱 마음까지 짱인 태권도장 사범. 얼굴만 봐도 신뢰감이 ...",NaN
2,강기영,1983,남,https://search.pstatic.net/common?type=b&size=...,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ...","['일에 대한 자부심이 강한 법무법인 한바다의 시니어 변호사이자 우영우의 멘토.',...",NaN
3,강남길,1958,남,https://search.pstatic.net/common?type=b&size=...,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과...",['산하의 아버지. 천성이 무골호인으로 허허실실 부드럽지만 처자식을 사랑하는 데는 ...,NaN
4,강동원,1981,남,https://search.pstatic.net/common?type=b&size=...,학창 시절 (강동원 졸업사진)중학교 때는 전교 2등을 할 정도로 공부를 굉장히 잘했...,NaN,NaN
...,...,...,...,...,...,...,...
1518,황채림,1996,여,https://search.pstatic.net/sunny/?src=https%3A...,"SUPER JUNIOR의 센터.[23] 최시원은 흔히 말하는 배우상, 즉 가장 정석...",NaN,NaN
1519,김명곤,1953,남,https://search.pstatic.net/sunny/?src=https%3A...,"1954년 출생[1]으로, 1972년 MBC 탤런트 공채 5기로 데뷔하여 현재까지 ...",NaN,NaN
1520,최시원,1986,남,https://search.pstatic.net/common?type=b&size=...,유명 가수 아빠 덕분에 어릴 때부터 많은 방송에 출연해 얼굴을 알렸다.판타지오의 연...,NaN,NaN
1521,김동주,1954,여,https://search.pstatic.net/common?type=b&size=...,아역배우 출신으로 MBC 드라마 '선덕여왕'에서 덕만 아역으로 대중들에게 각인된 배...,NaN,NaN


In [10]:
# 사망한 배우 제거
actor_df = actor_df[~(actor_df['나무위키']=='사망')].reset_index(drop=True)
actor_df

,Name,age,Sex,photo,나무위키,배역내용,커뮤니티
0,감우성,1970,남,https://search.pstatic.net/common?type=b&size=...,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,...",['첫사랑 수진과 열렬한 연애 끝에 결혼까지 성공했다. 남에게 피해 주지 않고 올곧...,NaN
1,강경준,1983,남,https://search.pstatic.net/common?type=b&size=...,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ...","['흙수저? 아니, 몸짱 얼짱 마음까지 짱인 태권도장 사범. 얼굴만 봐도 신뢰감이 ...",NaN
2,강기영,1983,남,https://search.pstatic.net/common?type=b&size=...,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ...","['일에 대한 자부심이 강한 법무법인 한바다의 시니어 변호사이자 우영우의 멘토.',...",NaN
3,강남길,1958,남,https://search.pstatic.net/common?type=b&size=...,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과...",['산하의 아버지. 천성이 무골호인으로 허허실실 부드럽지만 처자식을 사랑하는 데는 ...,NaN
4,강동원,1981,남,https://search.pstatic.net/common?type=b&size=...,학창 시절 (강동원 졸업사진)중학교 때는 전교 2등을 할 정도로 공부를 굉장히 잘했...,NaN,NaN
...,...,...,...,...,...,...,...
1514,황채림,1996,여,https://search.pstatic.net/sunny/?src=https%3A...,"SUPER JUNIOR의 센터.[23] 최시원은 흔히 말하는 배우상, 즉 가장 정석...",NaN,NaN
1515,김명곤,1953,남,https://search.pstatic.net/sunny/?src=https%3A...,"1954년 출생[1]으로, 1972년 MBC 탤런트 공채 5기로 데뷔하여 현재까지 ...",NaN,NaN
1516,최시원,1986,남,https://search.pstatic.net/common?type=b&size=...,유명 가수 아빠 덕분에 어릴 때부터 많은 방송에 출연해 얼굴을 알렸다.판타지오의 연...,NaN,NaN
1517,김동주,1954,여,https://search.pstatic.net/common?type=b&size=...,아역배우 출신으로 MBC 드라마 '선덕여왕'에서 덕만 아역으로 대중들에게 각인된 배...,NaN,NaN


In [16]:
# nan 값 모두 빈 문자열로 채우기
actor_df = actor_df.fillna(value='')

In [17]:
# 나무위키 배역내용 커뮤니티를 합친 새로운 columns 생성
content_list = []

for i in tqdm(range(len(actor_df))):
    temp_str = ''
    temp_str = actor_df['나무위키'][i] + actor_df['배역내용'][i] + actor_df['커뮤니티'][i]
    content_list.append(temp_str)

  0%|          | 0/1519 [00:00<?, ?it/s]

In [19]:
actor_df['total_content'] = content_list

In [21]:
# 필요없는 열 제거
actor_df = actor_df[['Name', 'age', 'Sex', 'total_content']]
actor_df

,Name,age,Sex,total_content
0,감우성,1970,남,"MBC 20기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규, 박철, 차광수,..."
1,강경준,1983,남,"초등학교 시절에는 야구, 중학교 시절에는 농구선수였었다. 농구를 소재로 한 드라마 ..."
2,강기영,1983,남,"tvN 드라마 고교처세왕, 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다. ..."
3,강남길,1958,남,"서울수색초등학교, 서울양정중학교, 서울대신고등학교를 졸업했고 한양대학교 영어영문학과..."
4,강동원,1981,남,학창 시절 (강동원 졸업사진)중학교 때는 전교 2등을 할 정도로 공부를 굉장히 잘했...
...,...,...,...,...
1514,황채림,1996,여,"SUPER JUNIOR의 센터.[23] 최시원은 흔히 말하는 배우상, 즉 가장 정석..."
1515,김명곤,1953,남,"1954년 출생[1]으로, 1972년 MBC 탤런트 공채 5기로 데뷔하여 현재까지 ..."
1516,최시원,1986,남,유명 가수 아빠 덕분에 어릴 때부터 많은 방송에 출연해 얼굴을 알렸다.판타지오의 연...
1517,김동주,1954,여,아역배우 출신으로 MBC 드라마 '선덕여왕'에서 덕만 아역으로 대중들에게 각인된 배...


In [23]:
# 한글 외 데이터 제거
actor_df['total_content'] = actor_df['total_content'].str.replace('[^가-힣 ]', '')

/tmp/ipykernel_4515/3477763608.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  actor_df['total_content'] = actor_df['total_content'].str.replace('[^가-힣 ]', '')
/tmp/ipykernel_4515/3477763608.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actor_df['total_content'] = actor_df['total_content'].str.replace('[^가-힣 ]', '')


In [24]:
# 형태소 토큰화
okt = Okt()

content_tag = []

In [25]:
for content in tqdm(actor_df['total_content']):
    morph = okt.pos(content, stem=True)
    content_tag.append(morph)

  0%|          | 0/1519 [00:00<?, ?it/s]

In [30]:
# 형용사만 추출
content_bucket_list = []

for content in tqdm(content_tag):
    for word, tag in content:
        if tag in ['Adjective']:
            content_bucket_list.append(word)

  0%|          | 0/1519 [00:00<?, ?it/s]

In [32]:
# 불용어 선정
content_counts = Counter(content_bucket_list)
content_counts.most_common(50)

[('있다', 8232),
 ('같다', 2220),
 ('없다', 2146),
 ('아니다', 1913),
 ('많다', 1846),
 ('좋다', 1668),
 ('그렇다', 980),
 ('좋아하다', 774),
 ('유명하다', 524),
 ('수상하다', 519),
 ('상당하다', 469),
 ('이렇다', 444),
 ('어떻다', 405),
 ('높다', 404),
 ('인하다', 360),
 ('예쁘다', 326),
 ('꾸준하다', 323),
 ('성공하다', 322),
 ('다양하다', 321),
 ('비슷하다', 304),
 ('굉장하다', 291),
 ('당하다', 278),
 ('강하다', 261),
 ('뛰어나다', 248),
 ('엄청나다', 247),
 ('젊다', 247),
 ('작다', 233),
 ('밉다', 227),
 ('친하다', 217),
 ('귀엽다', 214),
 ('다르다', 200),
 ('깊다', 196),
 ('힘들다', 195),
 ('희다', 195),
 ('새롭다', 175),
 ('의하다', 169),
 ('미치다', 166),
 ('어렵다', 165),
 ('잘생기다', 163),
 ('아쉽다', 163),
 ('괜찮다', 158),
 ('완벽하다', 158),
 ('나쁘다', 148),
 ('짧다', 148),
 ('심하다', 144),
 ('훌륭하다', 144),
 ('재밌다', 144),
 ('이쁘다', 134),
 ('착하다', 131),
 ('가깝다', 125)]

In [33]:
stopwords = ['있다','어느덧다','더하다', '재탕하다', '뜨겁다', '불가하다', '없다', '많다', '아니다', '같다','부다', '좋다', '그렇다', '상당하다', '다양하다', '높다', '인하다', '이렇다', '어떻다', '아쉽다', '당하다', '비슷하다', '의하다', '심하다', '아깝다', '빠르다', '가능하다', '싫다', '친하다', '당하다', '다르다', '괜찮다', '궁금하다', '안되다', '이다', '이렇다', '어떻다', '많다', '그렇다', '스럽다', '인하다', '원하다', '넘치다', '깊다', '똑같다', '멀다', '높다', '계시다', '심하다', '박하다', '상관없다', '안녕하다', '역다', '저렇다', '딱이다', '뿐이다', '쓰리다', '톡톡하다', '못지않다' ]

In [34]:
# 선정된 불용어 확인
stopwords = set(stopwords) # 중복 제거
stopwords

{'가능하다',
 '같다',
 '계시다',
 '괜찮다',
 '궁금하다',
 '그렇다',
 '깊다',
 '넘치다',
 '높다',
 '다르다',
 '다양하다',
 '당하다',
 '더하다',
 '딱이다',
 '똑같다',
 '뜨겁다',
 '많다',
 '멀다',
 '못지않다',
 '박하다',
 '부다',
 '불가하다',
 '비슷하다',
 '빠르다',
 '뿐이다',
 '상관없다',
 '상당하다',
 '스럽다',
 '싫다',
 '심하다',
 '쓰리다',
 '아깝다',
 '아니다',
 '아쉽다',
 '안녕하다',
 '안되다',
 '어느덧다',
 '어떻다',
 '없다',
 '역다',
 '원하다',
 '의하다',
 '이다',
 '이렇다',
 '인하다',
 '있다',
 '재탕하다',
 '저렇다',
 '좋다',
 '친하다',
 '톡톡하다'}

In [35]:
# 형용사만 추출 - 불용어 제외
content_total_keywords = []

for content in tqdm(content_tag):
    content_bucket_list = []
    for word, tag in content:
        if tag in ['Adjective']:
            if word not in stopwords:
                content_bucket_list.append(word)
    content_bucket_list = list(set(content_bucket_list))
    content_total_keywords.append(", ".join(content_bucket_list))

  0%|          | 0/1519 [00:00<?, ?it/s]

In [36]:
content_total_keywords[:100]

['성공하다, 예민하다, 평범하다, 후줄근하다, 아름답다, 강하다, 멋지다, 갑갑하다, 행복하다, 거대하다, 자유롭다, 특별하다, 여전하다, 수상하다, 예리하다, 진하다, 반하다, 작다, 자세하다, 무리하다, 엄청나다, 뛰어나다, 중요하다, 좋아하다, 성장하다, 따뜻하다, 미치다, 애절하다, 독하다, 새롭다, 어느새다, 까맣다, 꾸준하다, 식상하다, 열렬하다',
 '탄탄하다, 쿨하다, 다부지다, 좋아하다, 서글서글하다, 신기하다, 건강하다, 미끈하다',
 '밉다, 강하다, 유머러스하다, 유일하다, 좋아하다, 어렵다, 위대하다',
 '젊다, 부드럽다, 괴팍하다, 몰두하다, 무책임하다, 당연하다, 온화하다, 쉽다, 꼬장꼬장하다, 낡다, 무능하다, 뒤늦다, 지독하다, 우스꽝스럽다, 가난하다, 애틋하다',
 '유쾌하다, 힘들다, 평범하다, 소홀하다, 독하다, 따뜻하다, 좋아하다, 까맣다, 흔하다, 적응하다, 어렵다, 힘드다, 우수하다, 유일하다, 그을리다, 굉장하다',
 '젊다, 안타깝다, 찌질하다, 힘없다, 아름답다, 밉다, 잘생기다, 착하다, 무섭다',
 '희다, 바쁘다, 특이하다, 자리다, 망하다, 유일하다, 꾸준하다, 출중하다, 낮다, 위험하다, 나쁘다',
 '찌질하다',
 '좋아하다, 맵다',
 '부드럽다, 차분하다, 휘하다, 비열하다, 짧다, 강경하다',
 '엄청나다, 유명하다, 끊임없다, 어색하다, 영원하다, 나쁘다, 독실하다',
 '',
 '작다',
 '',
 '독특하다, 뻔하다, 친절하다, 신기하다, 힘드다, 출중하다, 비장하다, 깔끔하다, 나쁘다',
 '',
 '성공하다, 당연하다, 철저하다, 아름답다, 우월하다, 수상하다, 확실하다, 굳다, 어렵다, 편하다, 뛰어나다, 좋아하다, 탁월하다, 굉장하다, 휩쓸다, 완벽하다, 새롭다, 유치하다, 소중하다',
 '차분하다, 잘생기다, 미워하다, 묵묵하다, 드물다, 까맣다, 민감하다, 굵다',
 '당당하다, 귀엽다, 차갑다, 강하다, 딱딱하다, 불과하다, 성장하다',
 '수상하다, 붉다, 기이하다, 놀랍다,

In [38]:
actor_df['adjective'] = content_total_keywords
actor_df

/tmp/ipykernel_4515/3753137407.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actor_df['adjective'] = content_total_keywords


,Name,age,Sex,total_content,adjective
0,감우성,1970,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,"성공하다, 예민하다, 평범하다, 후줄근하다, 아름답다, 강하다, 멋지다, 갑갑하다,..."
1,강경준,1983,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,"탄탄하다, 쿨하다, 다부지다, 좋아하다, 서글서글하다, 신기하다, 건강하다, 미끈하다"
2,강기영,1983,남,드라마 고교처세왕 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다 대중들에게...,"밉다, 강하다, 유머러스하다, 유일하다, 좋아하다, 어렵다, 위대하다"
3,강남길,1958,남,서울수색초등학교 서울양정중학교 서울대신고등학교를 졸업했고 한양대학교 영어영문학과를 ...,"젊다, 부드럽다, 괴팍하다, 몰두하다, 무책임하다, 당연하다, 온화하다, 쉽다, 꼬..."
4,강동원,1981,남,학창 시절 강동원 졸업사진중학교 때는 전교 등을 할 정도로 공부를 굉장히 잘했었다 ...,"유쾌하다, 힘들다, 평범하다, 소홀하다, 독하다, 따뜻하다, 좋아하다, 까맣다, 흔..."
...,...,...,...,...,...
1514,황채림,1996,여,의 센터 최시원은 흔히 말하는 배우상 즉 가장 정석 미남형에 가까운 얼굴을 가지고...,"반듯하다, 유명하다, 차분하다, 굵직하다, 다분하다, 예쁘다, 당연하다, 성숙하다,..."
1515,김명곤,1953,남,년 출생으로 년 탤런트 공채 기로 데뷔하여 현재까지 중견 배우로 활동 중이다 년에...,
1516,최시원,1986,남,유명 가수 아빠 덕분에 어릴 때부터 많은 방송에 출연해 얼굴을 알렸다판타지오의 연습...,
1517,김동주,1954,여,아역배우 출신으로 드라마 선덕여왕에서 덕만 아역으로 대중들에게 각인된 배우 년 영...,"희다, 뛰어나다, 수상하다, 똑똑하다, 귀엽다, 나란하다, 철저하다, 오죽하다, 유..."


In [41]:
# 형용사가 없는 배우 제거
actor_df = actor_df[~(actor_df['adjective'] == '')].reset_index(drop=True)
actor_df

,Name,age,Sex,total_content,adjective
0,감우성,1970,남,기 공채 탤런트로 배우 생활을 시작했으며 동기로 한석규 박철 차광수 곽진영 김소이...,"성공하다, 예민하다, 평범하다, 후줄근하다, 아름답다, 강하다, 멋지다, 갑갑하다,..."
1,강경준,1983,남,초등학교 시절에는 야구 중학교 시절에는 농구선수였었다 농구를 소재로 한 드라마 마지...,"탄탄하다, 쿨하다, 다부지다, 좋아하다, 서글서글하다, 신기하다, 건강하다, 미끈하다"
2,강기영,1983,남,드라마 고교처세왕 오 나의 귀신님에서 감초역할로 등장하며 이름을 알렸다 대중들에게...,"밉다, 강하다, 유머러스하다, 유일하다, 좋아하다, 어렵다, 위대하다"
3,강남길,1958,남,서울수색초등학교 서울양정중학교 서울대신고등학교를 졸업했고 한양대학교 영어영문학과를 ...,"젊다, 부드럽다, 괴팍하다, 몰두하다, 무책임하다, 당연하다, 온화하다, 쉽다, 꼬..."
4,강동원,1981,남,학창 시절 강동원 졸업사진중학교 때는 전교 등을 할 정도로 공부를 굉장히 잘했었다 ...,"유쾌하다, 힘들다, 평범하다, 소홀하다, 독하다, 따뜻하다, 좋아하다, 까맣다, 흔..."
...,...,...,...,...,...
1358,황정민,1969,여,년 월 걸그룹 슈가로 데뷔했다 년 탈퇴 후 배우로 전업하였으며 년 우리 결혼했어요...,"성공하다, 동떨어지다, 발랄하다, 수상하다, 확실하다, 드물다, 밉다, 훌륭하다, ..."
1359,황정음,1984,여,년 월 일 전라북도 전주시 완산구에서 태어났다 전주초등학교와 전주고등학교 서울대학교...,수상하다
1360,황채림,1996,여,의 센터 최시원은 흔히 말하는 배우상 즉 가장 정석 미남형에 가까운 얼굴을 가지고...,"반듯하다, 유명하다, 차분하다, 굵직하다, 다분하다, 예쁘다, 당연하다, 성숙하다,..."
1361,김동주,1954,여,아역배우 출신으로 드라마 선덕여왕에서 덕만 아역으로 대중들에게 각인된 배우 년 영...,"희다, 뛰어나다, 수상하다, 똑똑하다, 귀엽다, 나란하다, 철저하다, 오죽하다, 유..."


In [50]:
actor_df.to_excel('./text_data/Actor_adjective_text.xlsx', encoding='utf-8-sig', index=False)

In [43]:
actor_df['adjective'].count()

1363